# <center>本地微调开源大模型

## <center>LLaMA-Factory项目介绍及微调实践

> **注意：大模型技术正处于快速发展阶段，因此在实际使用中可能会遇到与内容不一致的情况。当前课件内容截至时间时间为2024年09月18日。**

&emsp;&emsp;大模型的区别，我们介绍的都是基于Chat类模型的P-Turning或者LoRA微调方法，但大家需要明确的一点是：对于大语言模型来说，`Chat`类模型通常是在`Base`类模型的基础上，通过对话格式的微调或特殊训练得到的。基础模型（Base models）首先在大规模的文本数据上进行预训练，学习语言的基本规律、知识和语境理解能力。在这一阶段，大模型通常不会特别针对对话任务进行优化，其根本的目的还是要去学习广泛的语言表达和理解能力。然后，为了使模型更适合特定的任务，如聊天对话，这些基础模型会经过进一步的微调（fine-tuning），这往往采用的是全量微调的方法而不是所谓的高效微调。在全量微调阶段，大模型在对话数据集上进行训练，学习如何在对话上下文中生成连贯、相关且自然的回答。这一过程可能包括对特定的对话结构、回答风格或特定领域知识的优化。这种基于基础模型的微调方法使得`Chat`类模型能够在保持底层语言理解和生成能力的同时，更好地适应对话式的交互需求。

&emsp;&emsp;所以，我们才会经常见到的是，每一个特定尺寸的基础模型（Base model）往往会有一个对应尺寸的专为对话优化的模型（Chat model）。这种搭配主要是因为在实际应用中，对话优化的模型由于其对交流细节的优化，更能满足实际使用需求。然而，这并不意味着基础模型不能被微调来适应特定任务。事实上，通过选用合适的数据集和采用适当的训练策略，也完全可以在基础模型的框架内实现对人类意图的精准把握和响应。

<div align=center><img src="img/模型区别.png" width=80%></div>

&emsp;&emsp;不同的模型，无论是基础模型还是专门的对话模型，由于它们各自独特的训练数据格式和模型配置，深入了解并执行微调工作通常需要投入相当的时间。目前，许多项目旨在简化模型使用和训练过程，通过提供统一的集成解决方案帮助用户更快上手。我们将探讨一个出色的开源项目——LLaMA-Factory，它旨在为集成各种大模型微调任务，提供一种快速且便捷的通用使用方式。

# 1. LLaMA-Factory项目介绍

&emsp;&emsp;LLaMA Factory是一个在GitHub上开源的项目，该项目给自身的定位是：提供一个易于使用的大语言模型（LLM）微调框架，支持LLaMA、Baichuan、Qwen、ChatGLM等架构的大模型。更细致的看，该项目提供了从预训练、指令微调到RLHF阶段的开源微调解决方案。截止目前（2024年09月18日）支持约120+种不同的模型和内置了60+的数据集，同时封装出了非常高效和易用的开发者使用方法。而其中最让人喜欢的是其开发的LLaMA Board，这是一个零代码、可视化的一站式网页微调界面，它允许我们通过Web UI轻松设置各种微调过程中的超参数，且整个训练过程的实时进度都会在Web UI中进行同步更新。

&emsp;&emsp;简单理解，通过该项目我们只需下载相应的模型，并根据项目要求准备符合标准的微调数据集，即可快速开始微调过程，而这样的操作可以有效地将特定领域的知识注入到通用模型中，增强模型对特定知识领域的理解和认知能力，以达到“通用模型到垂直模型的快速转变”。

&emsp;&emsp;LLaMA Factory的GitHub地址如下：https://github.com/hiyouga/LLaMA-Factory/tree/main

<div align=center><img src="img/logo.png" width=80%></div>

&emsp;&emsp;在使用该项目进行微调的时候，我们**需要关注如下三点内容：该项目支持的大模型、支持的训练方法和对应的训练数据格式。**截止2024年09月18日，LLaMA Factory项目相关的更新和集成情况如下：

## 当前版本新增内容为：

Qwen2-VL 模型的微调、Liger Kernel使用 enable_liger_kernel: true 来加速训练、Adam-mini 优化器


Adam-mini
Adam-mini是一种优化器，旨在减少内存使用并提高训练效率。它是对传统Adam优化器的改进版本，主要通过以下方式实现内存节省和性能提升：
```json
参数分块：Adam-mini将模型参数按照Hessian矩阵的结构划分为多个块，每个块代表一组参数。这样可以减少需要独立学习率的数量。
块内平均学习率：对于每个参数块，Adam-mini不再为每个参数单独分配学习率，而是为整个块分配一个平均的学习率。这显著减少了所需的学习率数量。
内存节省：由于减少了学习率的数量，所需的二阶动量（即Adam中的v参数）的存储也相应减少。实验表明，这种方法可以减少45%到50%的内存占用，同时保持或提升模型性能。
Adam-mini在各种深度学习任务中表现出色，特别是在训练大型语言模型时，可以显著减少内存开销并提高训练速度。例如，在Llama2-7B模型的预训练中，Adam-mini实现了比AdamW高49.6%的吞吐量，并节省了33%的训练时间。
```
Liger-Kernel
Liger-Kernel是由LinkedIn开发的一组高效Triton内核，专为大型语言模型（LLM）训练设计。其主要作用包括：
```json
提高训练吞吐量：Liger-Kernel可以有效地将多GPU训练的吞吐量提高20%。这意味着在相同的时间内，可以处理更多的数据，从而加快训练过程。
减少内存使用：Liger-Kernel能够将内存使用量减少60%，这使得在相同的硬件条件下，可以训练更大的模型或使用更大的批量大小。
兼容性：Liger-Kernel与Flash Attention、PyTorch FSDP和Microsoft DeepSpeed等框架兼容，用户可以方便地将其集成到现有的训练管道中。
Liger-Kernel通过优化内核的计算效率和内存管理，使得大型语言模型的训练更加高效和经济。这对于需要处理大规模数据和复杂模型的研究和应用具有重要意义。
```
```json
总结来说，Adam-mini主要通过优化器的改进来减少内存使用和提高训练效率，而Liger-Kernel则通过高效的内核设计来提升多GPU训练的吞吐量和减少内存使用。这
两个项目在各自的领域内都提供了显著的性能提升，适用于大型语言模型的训练任务。
```

&emsp;&emsp;LLaMA-Factory目前支持微调的模型及对应的参数量：

|模型名|模型大小|Template|
|-----|-------|--------|
|Baichuan 2|7B/13B|baichuan2|
|BLOOM/BLOOMZ|560M/1.1B/1.7B/3B/7.1B/176B|-|
|ChatGLM3|6B|chatglm3|
|Command R|35B/104B|cohere|
|DeepSeek (Code/MoE)|7B/16B/67B/236B|deepseek|
|Falcon|7B/11B/40B/180B|falcon|
|Gemma/Gemma 2/CodeGemma|2B/7B/9B/27B|gemma|
|GLM-4|9B|glm4|
|InternLM2/InternLM2.5|7B/20B|intern2|
|Llama|7B/13B/33B/65B|-|
|Llama 2|7B/13B/70B|llama2|
|Llama 3/Llama 3.1|8B/70B|llama3|
|LLaVA-1.5|7B/13B|llava|
|MiniCPM|1B/2B/4B|cpm/cpm3|
|Mistral/Mixtral|7B/8x7B/8x22B|mistral|
|OLMo|1B/7B|-|
|PaliGemma|3B|paligemma|
|Phi-1.5/Phi-2|1.3B/2.7B|-|
|Phi-3|4B/7B/14B|phi|
|Qwen/Qwen1.5/Qwen2 (Code/Math/MoE)|0.5B/1.5B/4B/7B/14B/32B/72B/110B|qwen|
|Qwen2-VL|2B/7B|qwen2_vl|
|StarCoder 2|3B/7B/15B|-|
|XVERSE|7B/13B/65B|xverse|
|Yi/Yi-1.5 (Code)|1.5B/6B/9B/34B|yi|
|Yi-VL|6B/34B|yi_vl|
|Yuan 2|2B/51B/102B|yuan|


注意：对于所有“基座”（Base）模型，template 参数可以是 default, alpaca, vicuna 等任意值。但“对话”（Instruct/Chat）模型请务必使用对应的模板。
务必在训练和推理时采用完全一致的模板。

&emsp;&emsp;更详细模型支持 LLaMA-Factory/src/llamafactory/extras/constants.py

&emsp;&emsp;更详细template LLaMA-Factory/src/llamafactory/data/template.py

&emsp;&emsp;可以看到，当前主流的开源大模型，包括ChatGLM3、Qwen的第一代以及最新的1.5测试版本，还有Biachuan2等，已经完全支持不同规模的参数量。针对LLaMA架构的系列模型，该项目已经基本实现了全面的适配。而其支持的训练方法，也主要围绕（增量）预训练、指令监督微调、奖励模型训练、PPO 训练和 DPO 训练展开，具体情况如下：

|方法|全参数训练|部分参数训练|LoRA|QLoRA|
|----|--------|----------|----|-----|
|预训练|✅|✅|✅|✅|
|指令监督微调|✅|✅|✅|✅|
|奖励模型训练|✅|✅|✅|✅|
|PPO 训练|✅|✅|✅|✅|
|DPO 训练|✅|✅|✅|✅|
|KTO 训练|✅|✅|✅|✅|
|ORPO 训练|✅|✅|✅|✅|
|SimPO 训练|✅|✅|✅|✅|

预训练（Pre-training）
```json
预训练是指在大规模、多样化的文本数据集上对深度学习模型进行初步训练的过程。这个过程旨在让模型学习到语言的统计规律、语义信息和上下文关系，为后续的特定
任务微调打下基础。预训练通常使用无监督学习方法，例如通过预测下一个词或填补被掩盖的词来训练模型。
```

指令监督微调（Instruction Supervised Fine-Tuning, SFT）
```json
指令监督微调是一种通过在由（指令，输出）对组成的数据集上进一步训练LLMs的过程。其目的是提高模型理解和执行各种任务指令的能力。这一过程弥补了预训练阶段
的不足，使模型能够更好地遵循人类指令，生成符合预期的输出。
```

奖励模型训练（Reward Model Training, RM）
```json
奖励模型训练的目的是通过人类反馈来优化模型的输出质量。具体来说，模型生成的文本会被人类评分，奖励模型根据这些评分进行训练，以量化输出的好坏程度。然
后，这些奖励信号用于进一步微调语言模型，使其生成的内容更符合人类的期望。
```

PPO 训练（Proximal Policy Optimization, PPO 近端策略优化）
```json
PPO是一种基于策略梯度的强化学习算法，旨在解决训练不稳定性和样本效率问题。PPO通过限制新旧策略之间的差异来稳定训练过程，避免策略更新过大导致的不稳定。
它在强化学习中被广泛应用于优化模型的策略，使其能够在复杂环境中获得最大化的累积奖励。
```

DPO 训练（Direct Preference Optimization, DPO 直接偏好优化）
```json
DPO是一种直接优化语言模型以生成符合人类偏好的输出的方法。与传统的RLHF（基于人类反馈的强化学习）不同，DPO不需要先拟合一个奖励模型，而是直接使用人类
偏好数据进行优化。这种方法简化了训练过程，提高了效率。
```
 
KTO 训练（Kahneman-Tversky Optimization, KTO 卡尼曼-特沃斯基优化）
```json
KTO是一种根据用户正向或负向反馈进行模型训练的方法。它基于行为经济学中的Kahneman-Tversky理论，通过用户的反馈来高效对齐模型的行为偏好，从而优化模型
的输出质量。
```

ORPO 训练（Optimized Reward Policy Optimization, ORPO  优化奖励政策优化）
```json
ORPO是一种优化奖励策略的训练方法，旨在通过优化奖励信号来提高模型的性能。具体细节可能涉及对奖励函数的调整和策略的优化，以确保模型在训练过程中能够更好
地学习和适应任务需求。
```
 
SimPO 训练（Simulated Preference Optimization, SimPO  模拟偏好优化 ）
```json
SimPO是一种模拟偏好优化的方法，通过模拟用户偏好数据来训练模型。这种方法可以在缺乏真实用户反馈的情况下，通过模拟数据来进行模型优化，从而提高模型的性
能和适应性。
```

- **预训练**

&emsp;&emsp;在大模型的早期阶段，也就是诞生的过程中，往往不会经过Supervised finetuning（监督式微调） 的过程，它在训练时仅仅是去学习大量的语言基本规律和知识理解，其训练数据是类似这种形式：
```json
新竹水仙宫位于新竹市北区长和宫的隔壁，是一间供奉水仙尊王的庙宇，为二殿二廊式建筑。庙中的神像原是供奉于长和宫的后殿竹。该庙于1998年12月29日公告为古
迹。 沿革 水仙宫兴建于1863年，两年后完工。庙中主神是新竹郊商崇祀的航海神，原供奉在长和宫后殿，后来随著市街发展而由铺户集资买下长和宫左侧店屋兴筑此庙
竹。该庙于2001年再度整修竹。 建筑特色 前殿正殿之木结构有对场作，不过不明显。为台湾现存之较早期对场作建筑

普济寺 (北京朝阳门) 普济寺，位于北京市朝阳区朝阳门外大街北侧，是一座汉传佛教寺院。现已无存。 简介 《北京寺庙历史资料》1936年北平寺庙登记中记载，普
济寺位于朝阳门外大街285号。北平市《1947年第二次寺庙总登记》记载该寺住持为昌和。 1949年下半年，北京市人民政府在朝阳门外大街建立北京市第九书报阅览
室。1950年11月1日，北京市人民政府决定将其扩建为北京市第六文化馆。1953年更名为关厢文化馆，馆址设在普济寺内，占地面积400多平方米，设有阅览室、游艺
室。1954年，关厢文化馆改归东郊区主管。1955年，北京市文化局将该馆定为工厂区重点馆。1958年，关厢文化馆与高碑店文化馆合并为朝阳区文化馆，并且迁入位于
小庄的新址。 普济寺位于朝阳门外大街北侧，天仙护国佑圣延寿宫以东。现已无存。

...
...
...
```

> 上述数据截取自：Wikipedia：https://huggingface.co/datasets/pleisto/wikipedia-cn-20230720-filtered （注：需要开启科学上网才可以访问）

&emsp;&emsp;预训练数据通常来自于多样化的文本来源，以覆盖尽可能广泛的主题、语境和语言风格。这些数据可以是结构化的文本，也可以是非结构化的，比如新闻文章、科学文献等等，对于基于这样训练数据训练得到的模型，很难听懂人类下达的具体指令从而去完成复杂的任务，更多的是使用其文本的生成能力。

&emsp;&emsp;我们所一直提及的微调，从参数规模来看，大体上可以分为**全参数微调和高效参数微调**。全参数微调通常以预训练模型的初始权重为基础，在特定数据集上继续训练，更新模型的所有参数。相比之下，高效参数微调旨在使用更少的资源来更新模型参数，这包括仅更新部分参数或通过对参数施加某些结构化约束来实现。后者就涵盖了上边提到的高效微调技术，如PEFT、LoRA和QLoRA等方法。

&emsp;&emsp;另一方面，如果按照在大模型哪个阶段使用微调，或者根据模型微调的目标来区分，也可以从提示微调、指令微调、有监督微调的方式来具体分类。

- **指令微调**

&emsp;&emsp;指令微调，是一种通过在由（指令，输出）对组成的数据集上进一步训练LLMs的过程。其中，指令代表模型的人类指令，输出代表遵循指令的期望输出。这个过程有助于弥合LLMs的下一个词预测目标与用户让LLMs遵循人类指令的目标之间的差距。它可以被视为有监督微调（Supervised Fine-Tuning，SFT）的一种特殊形式。但是，它们的目标依然有差别。SFT是一种使用标记数据对预训练模型进行微调的过程，以便模型能够更好地执行特定任务。而指令微调是一种通过在包括（指令，输出）对的数据集上进一步训练大语言模型（LLMs）的过程，以增强LLMs的能力和可控性。**指令微调的特殊之处在于其数据集的结构，即由人类指令和期望的输出组成的配对。这种结构使得指令微调专注于让模型理解和遵循人类指令。**

&emsp;&emsp;总的来说，指令微调是有监督微调的一种特殊形式，专注于通过理解和遵循人类指令来增强大语言模型的能力和可控性。其形式往往是这样的：

```json
  {
    "instruction": "列出一个应该在野营应急包中的7件物品。",
    "input": "",
    "output": "这是一个我们应该在野营应急包中放置的7件物品列表：\n\n1. 先进救援包：用于治疗划痕，创伤，烧伤，刀伤等紧急情况。\n2. 多用途刀具：可用于切割，锯木，开瓶，刮刀等。\n3. 手电筒和备用电池：用于夜间照明。\n4. 火种：火柴或打火机，用于生火取暖或烹饪。\n5. 压缩饼干和能量棒：紧急能量补给。\n6. 可充电无线收音机：用于接收天气预报或求救信号。\n7. 防水雨衣和毛毯：用于防止降雨和保暖。"
  },
  {
    "instruction": "写出狗和猫的两个不同之处。",
    "input": "",
    "output": "1. 物种不同：狗属于犬科,而猫属于猫科。它们在生物分类上就有区别。\n\n2. 行为不同：狗更依赖人类,更忠诚,而猫则更独立。例如狗更喜欢和主人玩耍互动，接受训练，而猫一般更享受独处和独立玩耍。"
  },
```

&emsp;&emsp;在上述数据中，其各字段意义如下：
- instruction：任务指令，不能为空。
- input：任务输入，可为空。如果不为空，项目内部处理训练数据时，会将 instruction、input 拼接在一起作为任务的输入。
- output：任务输出，不能为空。

> 该数据截取自`alpaca_zh_demo.json`数据，来源于GPT-4 使用 ChatGPT 从 Alpaca 翻译而来的中文提示生成中文指令跟踪数据，获取地址如下： /LLaMA-Factory/data/alpaca_zh_demo.js ，

&emsp;&emsp;在该项目中，也内置了非常多的标准数据集，整体分为预训练的数据集、指令微调数据集和偏好数据集。具体各个数据集的详细情况，大家可以在其GitHub官网查看：https://github.com/hiyouga/LLaMA-Factory/blob/main/README_zh.md

<div align=center><img src="数据集.png" width=40%></div>

&emsp;&emsp;最后且最关键的一点需特别指出：虽然LLaMA-Factory项目允许我们在120余种大模型中灵活选择并快速开启微调工作，但运行特定参数量的模型是否可行，仍然取决于本地硬件资源是否充足。因此，在选择模型进行实践前，大家必须仔细参照下表，结合自己的服务器配置来决定，以避免因硬件资源不足导致的内存溢出等问题。不同模型参数在不同训练方法下的显存占用情况如下：

<div align=center><img src="img/硬件配置.png" width=50%></div>

# 2. LLaMA-Factory私有化部署

&emsp;&emsp;作为GitHub上的开源项目，LLaMA-Factory的本地私有化部署过程与我们之前介绍的大模型部署大体相同，主要包括创建Python虚拟环境、下载项目文件及安装所需的依赖包。这一过程相对直观。但在开始部署之前，我们需要先了解LLaMA-Factory私有化部署对本地软硬件环境的具体要求：

|必需项|至少|推荐|
|-----|----|---|
|python|3.8 |3.11|
|torch|1.13.1 |2.4.0|
|transformers |4.41.2|4.43.4|
|datasets|2.16.0 |2.20.0|
|accelerate|0.30.1 |0.32.0|
|peft|0.11.1 |0.12.0|
|trl|0.8.6 |0.9.6|

|可选项|至少|推荐|
|-----|----|---|
|CUDA|11.6 |12.2|
|deepspeed|0.10.0 |0.14.0|
|bitsandbytes|0.39.0 |0.43.1|
|vllm|0.4.3 |0.5.0|
|flash-attn|2.3.0 |2.6.3|

&emsp;&emsp;**注意：需要严格按照软硬件的版本要求进行环境配置，否则在使用时一定会出现额外的未知错误。**具体的LLaMA-Factory项目本地化部署过程如下：

- **Step 1. 检查NVIDIA显卡驱动**
```shell
nvidia-smi
```


&emsp;&emsp;按照对应的教程安装完成后，可以通过如下命令验证当前机器的驱动安装情况：

> 注：如果部署环境是租赁的云服务器，显卡驱动环境已经预设好，并不需要我们再次手动配置。

<div align=center><img src="img/显卡显示.png" width=100%></div>

&emsp;&emsp;如果可以正常显示不报错，说明NVIDIA显卡驱动安装成功。

- **Step 2. 检查Anaconda**

> 注：如果部署环境是租赁的云服务器，Anaconda环境已经预设好，并不需要我们再次手动配置。

&emsp;&emsp;通过如下命令验证当前环境的Conda：
```bash
conda --version
```

<div align=center><img src="img/condaver.png" width=100%></div>

&emsp;&emsp;如果可以正常输出Conda的版本，则说明Anaconda安装成功。

- **Step 3. 使用Conda创建LLaMA-Factory的Python虚拟环境**

&emsp;&emsp;我们需要借助Conda包版本工具，为LLaMA-Factory项目创建一个新的Python虚拟运行环境，执行代码如下：
```bash
 conda create  --name llama_factory python==3.11
```

<div align=center><img src="img/condacre.png" width=80%></div>

&emsp;&emsp;如上所示，新创建了一个名为LLaMA-Factory的Python虚拟环境，其Python版本为3.11。创建完成后，通过如下命令进入该虚拟环境，执行后续的操作：
```bash
conda activate llama_factory
```

<div align=center><img src="img/condaerro.png" width=80%></div>


异常处理 
```bash
source activate
再次执行
conda activate llama_factory
```

<div align=center><img src="img/condall.png" width=80%></div>

- **Step 4. 根据CUDA版本要求安装Pytorch：https://pytorch.org/get-started/previous-versions/**

安装pytorch

官网地址：https://pytorch.org/get-started/previous-versions/

&emsp;&emsp;这里根据自己电脑显卡驱动的CUDA版本要求，在Pytorch官网中找到适合自己的Pytorch安装命令。
```bash
    conda install pytorch==2.3.1 torchvision==0.18.1 torchaudio==2.3.1 pytorch-cuda=12.1 -c pytorch -c nvidia

```

<div align=center><img src="img/pytorch.png" width=80%></div>

&emsp;&emsp;复制安装命令，进入虚拟环境的终端中执行安装。注意：这个终端并不是查看`nvidia-smi`的windows终端，而是我们创建虚拟环境的conda终端。

<div align=center><img src="https://muyu001.oss-cn-beijing.aliyuncs.com/img/image-20240227224141100.png" width=80%></div>

- **Step 5. 验证安装GPU 版Pytorch是否成功**

&emsp;&emsp;待安装完成后，如果想要检查是否成功安装了GPU版本的PyTorch，可以通过几个简单的步骤在Python环境中进行验证：
```bash
   python -c "import torch; print(torch.__version__)"
   python -c "import torch;print(torch.cuda.is_available())"
```

<div align=center><img src="img/torchsucc.png" width=80%></div>

&emsp;&emsp;如果输出是 True，则表示GPU版本的PyTorch已经安装成功并且可以使用CUDA，如果输出是False，则表明没有安装GPU版本的PyTorch，或者CUDA环境没有正确配置，此时根据教程，重新检查自己的执行过程。

- **Step 6. 下载LLaMA-Factory的项目文件**

&emsp;&emsp;进入LLaMA-Factory的官方Github，地址：https://github.com/hiyouga/LLaMA-Factory ， 在 GitHub 上将项目文件下载到有两种方式：克隆 (Clone) 和 下载 ZIP 压缩包。推荐使用克隆 (Clone)的方式。我们首先在GitHub上找到其仓库的URL。

<div align=center><img src="img/下载.png" width=80%></div>

&emsp;&emsp;在执行命令之前，需要先安装git软件包，执行命令如下：
```bash
sudo apt install git
```

<div align=center><img src="img/git.png" width=80%></div>

&emsp;&emsp;执行克隆命令，将LLaMA-Factory Github上的项目文件下载至本地的当前路径下，如下：
```bash
    git clone https://github.com/hiyouga/LLaMA-Factory.git
```


<div align=center><img src="img/gitcl.png" width=80%></div>
<div align=center><img src="img/gitcl2.png" width=80%></div>

- **Step 7. 使用pip安装LLaMA-Factory项目代码运行的项目依赖**



&emsp;&emsp; 进入 路径 cd LLaMA-Factory
在LLaMA-Factory中提供的 `requirements.txt`文件包含了项目运行所必需的所有 Python 包及其精确版本号。使用pip一次性安装所有必需的依赖，执行命令如下：

```bash
pip install -r requirements.txt -i https://pypi.tuna.tsinghua.edu.cn/simple/
```

<div align=center><img src="img/pipan.png" width=80%></div>

&emsp;&emsp;通过上述步骤就已经完成了LLaMA-Factory模型的完整私有化部署过程。

- **Step 8. 启动界面**

1、单卡启动  
```bash
CUDA_VISIBLE_DEVICES=0 python src/webui.py
```
<div align=center><img src="img/danka.png" width=80%></div>

界面展示  当前界面为单卡启动对应设备数量为1 
```bash
CUDA_VISIBLE_DEVICES=0,1 python src/webui.py  为多卡启动，会展示为2
```

<div align=center><img src="img/dankaweb.png" width=60%></div>

```bash
因当前为租赁服务器Autodl 需要在服务器端启动端口后，本地执行远端连接服务器
例如： ssh -CNg -L 6006:127.0.0.1:6006 root@region-11.autodl.com -p 26040 后输入密码即可
-CNg 是SSH的参数，分别表示压缩数据、后台运行和不执行远程命令。
-L 6006:127.0.0.1:6006 表示将本地的6006端口映射到远程服务器的6006端口。
root@region-11.autodl.com 是SSH登录的用户名和地址。
-p 26040 是SSH连接的端口
当前端口为 7860 需要将6006 更改为7860后访问，本地地址为http://localhost:7860即可
```
<div align=center><img src="img/local.png" width=60%></div>




- **Step 9. 参数介绍**

选择zh 为中文，选定后界面更改

<div align=center><img src="img/web2.png" width=100%></div>
<div align=center><img src="img/web3.png" width=100%></div>

参数讲解：
<div align=left><img src="img/weitiaoyou.png" width=60%></div>

```json 
AdamW优化器的学习率
学习率的作用
    控制步长：学习率决定了每次迭代时模型参数更新的幅度。较大的学习率会使参数更新较大，较小的学习率则会使参数更新较小。
    影响收敛速度：学习率过大可能导致模型在训练过程中震荡，甚至无法收敛；学习率过小则可能导致收敛速度过慢，训练时间延长。
    调大调小学习率的区别
调大学习率：
    优点：可以加快模型的收敛速度，减少训练时间。
    缺点：可能导致模型在训练过程中震荡，甚至发散，无法找到最优解。
调小学习率：
    优点：可以使模型更稳定地收敛，找到更优的解。
    缺点：收敛速度较慢，训练时间较长。
```
训练总轮数
```json 
训练总轮数的作用
    充分训练模型：更多的训练轮数可以使模型在训练数据上进行更多次的学习，从而更好地拟合数据。
    防止过拟合：过多的训练轮数可能导致模型在训练数据上过拟合，无法在测试数据上表现良好。
调大调小训练总轮数的区别
调大训练总轮数：
    优点：可以使模型有更多的机会学习训练数据，提升模型在训练数据上的表现。
    缺点：可能导致过拟合，训练时间增加。
调小训练总轮数：
    优点：可以减少训练时间，降低过拟合的风险。
    缺点：可能导致模型未能充分学习训练数据，表现不佳。

建议组合使用
高学习率 + 少训练轮数：适用于模型初期快速收敛，但需要注意防止震荡和发散。
低学习率 + 多训练轮数：适用于模型稳定收敛，找到更优解，但训练时间较长。

```
最大梯度裁剪范数

```json 
    是一种防止梯度爆炸的技术。在反向传播过程中，如果梯度的范数超过了预设的最大值，则会对梯度进行缩放，使其不超过这个最大值。
    较大值：允许较大的梯度更新，可能加快收敛速度，但也增加了梯度爆炸的风险。可能会使训练不稳定，导致模型训练无法收敛，训练过程无法继续
    较小值：限制梯度更新的幅度，减少梯度爆炸的风险，但可能导致收敛速度变慢
```
每个数据集的最大样本数
```json
    每个数据集的最大样本数是指在训练过程中，模型从数据集中选取的最大样本数量。这一参数决定了模型在每个训练周期（epoch）中能够看到的数据量。

    较大值：可以使模型在训练过程中看到更多的数据，从而提高模型的泛化能力和性能。然而，处理更多的数据需要更多的计算资源和时间。
    较小值：减少了训练数据量，可以加快训练速度并减少计算资源的需求，但可能导致模型过拟合于较小的数据集，降低泛化能力
```
梯度累积步数
```json
梯度累积步数是指在更新模型参数之前，累积多少个小批量（mini-batches）的梯度。具体来说，如果梯度累积步数为N，那么模型会在N个小批量数据
上计算梯度并累积，然后再进行一次参数更新。

数值大小的区别
小的梯度累积步数
    频繁更新：梯度累积步数较小时，模型参数更新频率较高。例如，梯度累积步数为1时，每个小批量数据处理后都会更新一次参数。
    显存占用少：由于每次只处理一个小批量的数据，显存占用较少，适合显存非常有限的环境。
    训练不稳定：较小的有效批量大小可能导致梯度估计不稳定，训练过程中的波动较大，可能需要更小的学习率来稳定训练。
大的梯度累积步数
    减少更新频率：梯度累积步数较大时，模型参数更新频率较低。例如，梯度累积步数为8时，每处理8个小批量数据后才更新一次参数。
    显存占用多：虽然每次处理的小批量数据显存占用不变，但累积的梯度需要更多的显存，因此总显存占用会增加。
    训练更稳定：较大的有效批量大小可以使梯度估计更稳定，从而提高训练的稳定性和收敛速度。适合需要高稳定性的任务。
    建议
        显存限制：根据显存大小选择合适的梯度累积步数。如果显存非常有限，可以选择较小的梯度累积步数；如果显存较为充裕，可以选择较大的梯度累积步数。
任务需求：根据具体任务和数据集的特点选择合适的梯度累积步数。对于需要高稳定性的任务，可以选择较大的梯度累积步数；对于快速迭代的任务，可以选择较小的梯度累积步
```
每个GPU处理的样本数量(batch size)
```json
    每个GPU处理的样本数量通常称为批量大小（Batch Size），是指在每次训练迭代中，每个GPU处理的数据样本数量。

    较大值：可以提高计算效率和硬件利用率，特别是在使用GPU进行并行计算时。然而，过大的批量大小可能导致显存不足，并且可能使模型训练不稳定。
    较小值：减少了每次迭代的显存需求，适用于显存较小的GPU，但可能导致训练时间延长和模型收敛速度变慢
```

截断长度
```json
    截断长度（Sequence Length/Max Token Length）是指在训练过程中，每个输入序列的最大长度。超过这个长度的输入序列将被截断或分割。

    内存管理：控制输入序列的长度可以有效管理内存使用，防止内存溢出。
    模型性能：适当的截断长度可以确保模型在处理长文本时的效率和性能。
选择建议
    任务需求：根据具体任务和数据集的特点选择合适的截断长度。例如，对于长文本处理任务，可以选择较大的截断长度。
    硬件资源：考虑硬件资源的限制，选择在内存允许范围内的最大截断长度
```

学习率调节器
```json
    学习率调节器（Learning Rate Scheduler）是一种动态调整学习率的机制，旨在优化训练过程中的学习率设置。
    优化收敛：通过动态调整学习率，可以避免训练过程中的过拟合或欠拟合，优化模型的收敛速度和效果。
    稳定训练：学习率调节器可以在训练过程中逐步降低学习率，防止梯度爆炸或梯度消失
```


# 3. LLaMA-Factory 微调


&emsp;&emsp;当前系统默认需要科学上网下载模型，如果科学上网不方便的话，还想通过在线的方式下载大模型，可以将模型下载源指向国内的ModelScope社区。这需要先中断当前的LLaMA-Board的后台服务重新启动，具体操作如下：
```bash
    当前我们选择多卡启动 首先进入LLaMA-Factory文件夹下
    # Linux操作系统， 如果是 Windows 使用 `set USE_MODELSCOPE_HUB=1`
    export USE_MODELSCOPE_HUB=1 
    CUDA_VISIBLE_DEVICES=0,1 python src/webui.py
```

- **Step 1. 下载模型**
根据
<div align=left><img src="img/qw.png" width=50%></div>
可以看到llama_factory支持的模型与对应的ID 可以去modelScop社区进行搜索下载 https://www.modelscope.cn/models/qwen/qwen2-1.5b-instruct/files
<div align=left><img src="img/下载mo.png" width=30%></div>

这里采取git 方式进行下载
```
创建路径
mkdir qwen
进入路径 
cd qwen
git lfs install
如果异常执行
sudo apt-get update
sudo apt-get install git-lfs

```
<div align=left><img src="img/git1.png" width=50%></div>
<div align=left><img src="img/git2.png" width=60%></div>

```json
再次执行
    git lfs install
下载文件
   git clone https://www.modelscope.cn/qwen/qwen2-1.5b-instruct.git
   
```

<div align=left><img src="img/git3.png" width=60%></div>




```bash
安装llama_factory 本身 进入LLaMA-Factory 根目录
pip install -e '.[metrics]'
```
<div align=left><img src="img/安装me.png" width=60%></div>

顺便说一下 llamafactory-cli 的参数说明

|动作参数枚举|参数说明|
|----------|-------|
|version|显示版本信息|
|train|命令行版本训练|
|chat|命令行版本推理chat|
|export|模型合并和导出|
|api|启动API server，供接口调用|
|eval|使用mmlu等标准数据集做评测|
|webchat|前端版本纯推理的chat页面|
|webui|启动LlamaBoard前端页面，包含可视|


多卡启动模型
```bash
CUDA_VISIBLE_DEVICES=0,1 llamafactory-cli webchat \
    --model_name_or_path /root/autodl-tmp/muyan/LLaMA-Factory/qwen/qwen2-1.5b-instruct \
    --template qwen
```
<div align=left><img src="img/qi1.png" width=60%></div>
界面访问：
<div align=left><img src="img/webqi1.png" width=60%></div>

```json
注意：当前模型启动端口界面与微调界面默认为同一端口地址(7860) 
    开启微调界面需要关闭当前模型访问界面
    开启微调界面需要关闭当前模型访问界面
    开启微调界面需要关闭当前模型访问界面
```



- **Step 2. 准备微调模型数据集合**

系统目前支持 alpaca 和sharegpt两种数据格式，以alpaca为例，整个数据集是一个json对象的list，具体数据格式为
```json
[
  {
    "instruction": "用户指令（必填）",
    "input": "用户输入（选填）",
    "output": "模型回答（必填）",
    "system": "系统提示词（选填）",
    "history": [
      ["第一轮指令（选填）", "第一轮回答（选填）"],
      ["第二轮指令（选填）", "第二轮回答（选填）"]
    ]
  }
]
```

例子比如单轮（alpaca_data_zh_51k.json 中的例子, 数据集在data/dataset_info.json中注册为alpaca_zh）
```json
{
  "instruction": "写一个有效的比较语句",
  "input": "篮球和足球",
  "output": "篮球和足球都是受欢迎的运动。"
}
```

和多轮 （oaast_sft_zh.json 中的例子, 数据集在data/dataset_info.json中注册为oaast_sft_zh）

```json
{
  "instruction": "谢谢",
  "input": "",
  "output": "不用谢! 很高兴我提供的信息能够帮助到你! 如果还有什么其他问题也可以向我提问。",
  "history": [
    [
      "请你给我写一个面试准备计划，我想要去面试微软的程序员岗位",
      "首先，你可以去微软官网寻找招聘信息并申请面试。\n其次，您可以在社交媒体平台寻找微软公司对程序员的面试问题，并做好准备。\n最后，您可以自己对面试过程进行模拟，熟悉话题并减少紧张感。\n我希望你能面试成功。"
    ]
  ]
}
```

所以我们训练的数据最好也转换成这种格式，然后在 data/dataset_info.json中进行注册（如果不做字段名称转换，则需要在注册的时候在 columns字段中做两个数据的映射配置）



首先进入数据路径 /home/muyan/LLaMA-Factory

```json
使用下面命令对文件中 name 与 author进行替换
sed -i 's/{{name}}/MuYan_yanxishe/g'  data/identity.json 
sed -i 's/{{author}}/muyan/g'  data/identity.json 
```
替换前
```json
{
  "instruction": "Who are you?",
  "input": "",
  "output": "Hello! I am {{name}}, an AI assistant developed by {{author}}. How can I assist you today?"
}
```
替换后
```json
{
  "instruction": "Who are you?",
  "input": "",
  "output": "I am MuYan_yanxishe, an AI assistant developed by muyan. How can I assist you today?"
}
```

当前为已有数据集变更，如果想要使用自己数据集需要将文件身上传到/data路径中，并在data/dataset_info.json将文件进行注册。

在之后的课程里我会对自定义数据集注册及使用进行解释，今天先试用已有数据集做流程验证。


- **Step 3. 启动LLamafactory 界面**

&emsp;&emsp;模型启动 加 export USE_MODELSCOPE_HUB=1(liunx\mac系统)
set USE_MODELSCOPE_HUB=1（window系统）

```json
linux 多卡启动命令
export USE_MODELSCOPE_HUB=1
    CUDA_VISIBLE_DEVICES=0,1 llamafactory-cli webui

通常会使用 nohup命令进行启动
export USE_MODELSCOPE_HUB=1
export CUDA_VISIBLE_DEVICES=0,1
nohup llamafactory-cli webui >20240919.log 2>&1 & 

```

&emsp;&emsp;按照上述配置修改大模型的下载源后，当执行微调时，便会从ModelScope社区先下载模型至本地，再开启正式的微调训练，这可以在后台中明显的看到下载的进程：


&emsp;&emsp;通过LLaMA Board在线下载至本地的模型存放路径为：`/root/.cache/huggingface/hub` 或者`/root/.cache/modelscope/hub/`下，这取决于从哪个官方执行的下载过程。当然，如果在本地已经下载好模型，在`Model path`中可以直接指定模型在服务器上的实际存储路径，比如我的本地服务器上已经下载好了`/home/muyan/LLaMA-Factory/qwen/qwen2-1.5b-instruct`，那么前端就可以这样配置：
<div align=left><img src="img/qwe1.png" width=60%></div>

其余参数配置
<div align=left><img src="img/wei1.png" width=60%></div>

&emsp;&emsp;不同的模型有不同的Prompt template，其定义的程序文件存放路径在：`LLaMA-Factory/src/llamafactory/data/template.py`

数据预览效果
<div align=left><img src="img/yulan.png" width=60%></div>

综合命令预览
<div align=left><img src="img/can1.png" width=60%></div>

- **点击开始**

点击开始 开始微调
<div align=left><img src="img/sta1.png" width=60%></div>

后端展示微调过程
<div align=left><img src="img/hou1.png" width=60%></div>

模型正常启动GPU使用状况
<div align=left><img src="img/yuan1.png" width=60%></div>

微调过程中GPU使用状况
<div align=left><img src="img/yuan2.png" width=60%></div>


训练完成
<div align=left><img src="img/sun1.png" width=60%></div>

加载模型
<div align=left><img src="img/jiazai.png" width=60%></div>

加载模型后进行测试 
输出`您好，我是 MuYan_yanxishe，一个由 muyan 开发的人工智能助手。我可以回答各种问题，提供实用的建议，帮助您解决问题。`
<div align=left><img src="img/ce1.png" width=60%></div>

微调成功！




&emsp;&emsp;除了上述核心配置选项之外，日志记录、模型保存等其他训练过程相关的细节也可以进行个性化设置。具体调整哪些参数如LoRA或RLHF，取决于我们之前选择的微调方法。例如，如果我们选择了LoRA微调，那么我们应进一步精细化LoRA的结构参数设置。调整RLHF相关参数则不会对训练产生影响，因为这一方法并未被使用。

<div align=center><img src="img/oth1.png" width=100%></div>

- **Step 4. 导出lora微调后模型**

创建导出路径`mkdir /home/muyan/LLaMA-Factory/new_qwen`

<div align=left><img src="img/daochu1.png" width=80%></div>

后端进入`cd /home/muyan/LLaMA-Factory/new_qwen `查看 
` ll `
<div align=left><img src="img/houqw1.png" width=80%></div>

- **启动合成后模型 **

关闭当前微调界面
```json
ps -ef | grep llama 
kill -9 
```
<div align=left><img src="img/kill1.png" width=80%></div>

启动合成后模型

```json
CUDA_VISIBLE_DEVICES=0,1 llamafactory-cli webchat \
    --model_name_or_path /root/autodl-tmp/muyan/LLaMA-Factory/2024_09_qwen \
    --template qwen
```


<div align=left><img src="img/houduan1.png" width=80%></div>

启动后界面刷新localhost:7860
<div align=left><img src="img/all1.png" width=80%></div>

```json

至此，模型lora形式微调、合并输出全部完成

```





除了上边常用微调，还可以看到关于参数保存及载入训练参数信息
<div align=left><img src="img/sav1.png" width=80%></div>

当前可以看到后端看到保存参数yaml文件
<div align=left><img src="img/houduansa.png" width=80%></div>

路径为 `/home/muyan/LLaMA-Factory/config`

可以使用载入训练参数重复使用当前参数
<div align=left><img src="img/zairu.png" width=80%></div>




-**命令详解**
```json
注意：当前命令中除了 绝对路径外均为项目根目录的相对路径 

llamafactory-cli train \
--stage sft     ##指定训练阶段为“sft”（supervised fine-tuning，监督微调）。
--do_train True    ##表示执行训练过程。
--model_name_or_path /home/muyan/LLaMA-Factory/qwen/qwen2-1.5b-instruct    ##指定预训练模型的路径或名称。
--preprocessing_num_workers 16    ##设置数据预处理时使用的工作线程数。
--finetuning_type lora    ##指定微调类型为LoRA（Low-Rank Adaptation）。
--template qwen    ##使用的模板名称，这里是“qwen”。
--flash_attn auto    ##设置闪存注意力机制，自动选择是否启用。
--dataset_dir data    ##数据集所在目录。
--dataset identity    ##使用的数据集名称。
--cutoff_len 1024    ##输入序列的最大长度。
--learning_rate 5e-05    ##学习率。
--num_train_epochs 3.0    ##训练的总轮数。
--max_samples 100000    ##最大样本数量。
--per_device_train_batch_size 2    ##每个设备上的训练批次大小。
--gradient_accumulation_steps 8    ##梯度累积步数。
--lr_scheduler_type cosine    ##学习率调度器类型，这里是余弦调度。
--max_grad_norm 1.0    ##梯度裁剪的最大范数。
--logging_steps 5    ##日志记录的步数间隔。
--save_steps 100    ##模型保存的步数间隔。
--warmup_steps 0    ##学习率预热步数。
--optim adamw_torch    ##优化器类型，这里是AdamW。
--packing False    ##是否启用数据打包。
--report_to none    ##日志报告的目标，这里设置为不报告。
--output_dir saves/Qwen2-1.5B-Chat/lora/train_2024-09-18-16-22-33    ##输出目录，用于保存训练结果。
--bf16 True    ##是否使用bfloat16精度。
--plot_loss True    ##是否绘制损失曲线。
--ddp_timeout 180000000    ##分布式数据并行（DDP）的超时时间。
--include_num_input_tokens_seen True    ##是否包含已处理的输入token数量。
--lora_rank 8    ##LoRA的秩。
--lora_alpha 16    ##LoRA的alpha参数。
--lora_dropout 0    ##LoRA的dropout率。
--lora_target all    ##LoRA的目标层。
```


# 4. 使用deepspeed 进行微调

&emsp;&emsp;在上一节中，我们直接使用了llama_factory 相关的多卡微调，1.5B模型消耗现存接近16G，对于更大模型微调资源消耗是非常可观的，所以这里可以使用deepspeed组件进行资源节省。



- **deepspeed**

&emsp;&emsp;DeepSpeed是一个由微软开发的开源深度学习优化库，旨在提高大规模模型训练和推理的效率和可扩展性。它主要用于分布式训练和优化深度学习模型，特别是那些具有数十亿甚至数万亿参数的超大规模模型。

&emsp;&emsp;
```json
核心技术之一是 ZeRO（Zero Redundancy Optimizer），它通过分阶段优化内存使用，使得在有限的硬件资源上训练超大规模模型成为可能。

ZeRO Stage 1
作用：优化器状态分区：ZeRO Stage 1 通过将优化器状态分割到各个进程中，每个进程只更新其分区，从而减少内存占用。
优势：内存节省：显著减少了每个 GPU 上的内存占用，使得可以在相同的硬件资源上训练更大规模的模型。

ZeRO Stage 2
作用：梯度分区：在 Stage 1 的基础上，ZeRO Stage 2 进一步将梯度分区到各个进程中，每个进程只保留和更新其分区的梯度。
优势：进一步内存优化：通过分区梯度，进一步减少了内存占用，使得可以训练更大规模的模型。
     提高计算效率：减少了梯度通信的开销，提高了分布式训练的效率。
ZeRO Stage 3
作用：参数分区：在 Stage 1 和 Stage 2 的基础上，ZeRO Stage 3 将模型参数也进行分区，每个进程只保留和更新其分区的参数。
优势：最大化内存节省：通过分区模型参数，最大限度地减少了内存占用，使得可以在有限的硬件资源上训练超大规模模型。
     支持超大规模模型：使得在现有硬件上训练和推理具有数万亿参数的模型成为可能。
ZeRO-Offload
作用：CPU 和 NVMe Offload：ZeRO-Offload 技术允许将优化器状态、梯度和模型参数从 GPU 内存中卸载到 CPU 内存或 NVMe 存储中，从而进一步减少 GPU 内存占用。
优势：显著降低 GPU 内存需求：
        通过将部分计算和存储任务卸载到 CPU 或 NVMe，显著减少了 GPU 内存需求，使得可以在单个 GPU 上训练更大规模的模型。
    提高硬件利用率：
      充分利用 CPU 和 NVMe 资源，提高了整体硬件利用率和训练效率。
总的来说，DeepSpeed 的 ZeRO 技术通过分阶段优化内存使用和计算效率，使得在有限的硬件资源上训练和推理超大规模模型成为可能。每个阶段和 Offload 技术都有其独特的作用和优势，适用于不同规模和需求的深度学习任务
```

**预览命令**
使用deepspeed stage3 offload模式
与上边命令相比只增加了` --deepspeed cache/ds_z3_offload_config.json`

<div align=left><img src="img/dep3.png" width=50%></div>


点击开始 报错
<div align=left><img src="img/weberr.png" width=50%></div>
后端显示 缺少 deepspeed 安装即可`注意上边所说deepspeed的版本推荐`

<div align=left><img src="img/houerrd.png" width=50%></div>

安装即可`pip3 install deepspeed==0.14.0`
<div align=left><img src="img/andeep.png" width=50%></div>





后端GPU使用监控
<div align=left><img src="img/dep3nvi.png" width=50%></div>
GPU使用对比未使用deepspeed offload功能
<div align=left><img src="img/duibi.png" width=50%></div>

微调完成 验证

<div align=left><img src="img/deep3yan.png" width=50%></div>

# 5. 注册自己的数据集

- **step 1. 识别数据集，并做数据准备**

dataset_info.json)包含了所有可用的数据集。如果您希望使用自定义数据集，请**务必**在 `dataset_info.json` 文件中添加*数据集描述*，并通过修改 `dataset: 数据集名称` 配置来使用数据集。

目前支持 **alpaca** 格式和 **sharegpt** 格式的数据集。
## sharegpt 格式

```json
"数据集名称": {
  "hf_hub_url": "Hugging Face 的数据集仓库地址（若指定，则忽略 script_url 和 file_name）",
  "ms_hub_url": "ModelScope 的数据集仓库地址（若指定，则忽略 script_url 和 file_name）",
  "script_url": "包含数据加载脚本的本地文件夹名称（若指定，则忽略 file_name）",
  "file_name": "该目录下数据集文件夹或文件的名称（若上述参数未指定，则此项必需）",
  "formatting": "数据集格式（可选，默认：alpaca，可以为 alpaca 或 sharegpt）",
  "ranking": "是否为偏好数据集（可选，默认：False）",
  "subset": "数据集子集的名称（可选，默认：None）",
  "split": "所使用的数据集切分（可选，默认：train）",
  "folder": "Hugging Face 仓库的文件夹名称（可选，默认：None）",
  "num_samples": "该数据集所使用的样本数量。（可选，默认：None）",
  "columns（可选）": {
    "prompt": "数据集代表提示词的表头名称（默认：instruction）",
    "query": "数据集代表请求的表头名称（默认：input）",
    "response": "数据集代表回答的表头名称（默认：output）",
    "history": "数据集代表历史对话的表头名称（默认：None）",
    "messages": "数据集代表消息列表的表头名称（默认：conversations）",
    "system": "数据集代表系统提示的表头名称（默认：None）",
    "tools": "数据集代表工具描述的表头名称（默认：None）",
    "images": "数据集代表图像输入的表头名称（默认：None）",
    "videos": "数据集代表视频输入的表头名称（默认：None）",
    "chosen": "数据集代表更优回答的表头名称（默认：None）",
    "rejected": "数据集代表更差回答的表头名称（默认：None）",
    "kto_tag": "数据集代表 KTO 标签的表头名称（默认：None）"
  },
  "tags（可选，用于 sharegpt 格式）": {
    "role_tag": "消息中代表发送者身份的键名（默认：from）",
    "content_tag": "消息中代表文本内容的键名（默认：value）",
    "user_tag": "消息中代表用户的 role_tag（默认：human）",
    "assistant_tag": "消息中代表助手的 role_tag（默认：gpt）",
    "observation_tag": "消息中代表工具返回结果的 role_tag（默认：observation）",
    "function_tag": "消息中代表工具调用的 role_tag（默认：function_call）",
    "system_tag": "消息中代表系统提示的 role_tag（默认：system，会覆盖 system column）"
  }
}
```

## Alpaca 格式

### 指令监督微调数据集

在指令监督微调时，`instruction` 列对应的内容会与 `input` 列对应的内容拼接后作为人类指令，即人类指令为 `instruction\ninput`。而 `output` 列对应的内容为模型回答。

如果指定，`system` 列对应的内容将被作为系统提示词。

`history` 列是由多个字符串二元组构成的列表，分别代表历史消息中每轮对话的指令和回答。注意在指令监督微调时，历史消息中的回答内容**也会被用于模型学习**。

```json
[
  {
    "instruction": "人类指令（必填）",
    "input": "人类输入（选填）",
    "output": "模型回答（必填）",
    "system": "系统提示词（选填）",
    "history": [
      ["第一轮指令（选填）", "第一轮回答（选填）"],
      ["第二轮指令（选填）", "第二轮回答（选填）"]
    ]
  }
]
```

对于上述格式的数据，`dataset_info.json` 中的*数据集描述*应为：

```json
"数据集名称": {
  "file_name": "data.json",
  "columns": {
    "prompt": "instruction",
    "query": "input",
    "response": "output",
    "system": "system",
    "history": "history"
  }
}
```

### 预训练数据集

在预训练时，只有 `text` 列中的内容会用于模型学习。

```json
[
  {"text": "document"},
  {"text": "document"}
]
```

对于上述格式的数据，`dataset_info.json` 中的*数据集描述*应为：

```json
"数据集名称": {
  "file_name": "data.json",
  "columns": {
    "prompt": "text"
  }
}
```

### 偏好数据集

偏好数据集用于奖励模型训练、DPO 训练、ORPO 训练和 SimPO 训练。

它需要在 `chosen` 列中提供更优的回答，并在 `rejected` 列中提供更差的回答。

```json
[
  {
    "instruction": "人类指令（必填）",
    "input": "人类输入（选填）",
    "chosen": "优质回答（必填）",
    "rejected": "劣质回答（必填）"
  }
]
```

对于上述格式的数据，`dataset_info.json` 中的*数据集描述*应为：

```json
"数据集名称": {
  "file_name": "data.json",
  "ranking": true,
  "columns": {
    "prompt": "instruction",
    "query": "input",
    "chosen": "chosen",
    "rejected": "rejected"
  }
}
```


- **step 2. 上传数据集**
<div align=left><img src="img/data1.png" width=50%></div>

将数据注册到 dataset_info.json
<div align=left><img src="img/data2.png" width=50%></div>


- **step 3. 界面使用数据集**
<div align=left><img src="img/data3.png" width=50%></div>

# 6. 多机分布运行

分布式运行命令
```json
FORCE_TORCHRUN=1 NNODES=2 RANK=0 MASTER_ADDR=192.168.0.1 MASTER_PORT=29500 llamafactory-cli train examples/train_lora/llama3_lora_sft.yaml
FORCE_TORCHRUN=1 NNODES=2 RANK=1 MASTER_ADDR=192.168.0.1 MASTER_PORT=29500 llamafactory-cli train examples/train_lora/llama3_lora_sft.yaml
```
```json
参数解释
FORCE_TORCHRUN=1：这个环境变量强制使用 torchrun 来启动分布式训练。torchrun 是 PyTorch 提供的一个用于分布式训练的工具。

NNODES=2：表示分布式训练中使用的节点（机器）数量为 2。

RANK=0 和 RANK=1：表示当前节点的排名（编号）。在分布式训练中，每个节点都有一个唯一的排名。这里 RANK=0 表示第一个节点，RANK=1 表示第二个节点。

MASTER_ADDR=192.168.0.1：表示主节点的 IP 地址。所有节点都会连接到这个主节点进行通信和同步。

MASTER_PORT=29500：表示主节点用于通信的端口号。所有节点都会通过这个端口进行通信。

llamafactory-cli train examples/train_lora/llama3_lora_sft.yaml：这是 LLaMA-Factory 的命令行接口，用于启动训练任务。train 表示训练任务，examples/train_lora/llama3_lora_sft.yaml 是训练配置文件的路径。

工作原理
启动分布式训练：

这两条命令分别在两台机器上执行，启动分布式训练任务。
每台机器上都会运行一个训练进程，并且这些进程会通过主节点的 IP 地址和端口进行通信和同步。
配置文件：

examples/train_lora/llama3_lora_sft.yaml 是训练配置文件，包含了训练所需的所有参数和设置，例如模型类型、数据集路径、训练超参数等。
同步和通信：

所有节点通过 MASTER_ADDR 和 MASTER_PORT 进行通信和同步，确保每个节点上的训练进程都能正确地协同工作。
示例
假设你有两台机器，IP 地址分别为 192.168.0.1 和 192.168.0.2，你需要在每台机器上分别运行以下命令：

在主节点（192.168.0.1）上运行：
FORCE_TORCHRUN=1 NNODES=2 RANK=0 MASTER_ADDR=192.168.0.1 MASTER_PORT=29500 llamafactory-cli train examples/train_lora/llama3_lora_sft.yaml
在从节点（192.168.0.2）上运行：
FORCE_TORCHRUN=1 NNODES=2 RANK=1 MASTER_ADDR=192.168.0.1 MASTER_PORT=29500 llamafactory-cli train examples/train_lora/llama3_lora_sft.yaml
```



